In [156]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow import keras

In [157]:
data = pd.read_csv('Data_folder/robo_coord_angle_dataset.csv')

In [158]:
data.isnull().any()

Unnamed: 0    False
x_coord       False
y_coord       False
z_coord       False
alpha         False
beta          False
gamma         False
dtype: bool

In [159]:
data.head(10)

,Unnamed: 0,x_coord,y_coord,z_coord,alpha,beta,gamma
0,0,12.524,-50.233,1432.786,104,-38,-90
1,1,7.796,-31.269,1432.445,104,-38,-88
2,2,3.074,-12.329,1431.422,104,-38,-86
3,3,-1.637,6.565,1429.718,104,-38,-84
4,4,-6.330,25.389,1427.336,104,-38,-82
5,5,-11.001,44.122,1424.278,104,-38,-80
6,6,-15.643,62.739,1420.549,104,-38,-78
7,7,-20.250,81.219,1416.152,104,-38,-76
8,8,-24.818,99.539,1411.093,104,-38,-74
9,9,-29.340,117.677,1405.378,104,-38,-72


In [160]:
X_df = data.iloc[:, 1:4]
Y_df = data.iloc[:, 4:]
X = X_df.values
Y = Y_df.values

In [161]:
#splitting the dataset into train, valid, test in the ratio  of 6:2:2
Xtrainval, Xtest, Ytrainval, Ytest = train_test_split(X, Y, test_size=0.2, random_state = 42)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrainval, Ytrainval, test_size=0.16, random_state = 42)

In [162]:
#scaling (standard) X data as per Xtrain
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)

In [163]:
from keras import backend as K
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [164]:
#model
def model_development(neurons, layrs, gap) :
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal', input_shape=Xtrain.shape[1:]))
    if layrs <= 5:
        for i in range(layrs-1):
            model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
    else:
        flag = 0
        for i in range(0,layrs,gap+1):
            if flag == 1:
                for j in range(gap):
                    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
            else:
                for j in range(gap-1):
                    model.add(keras.layers.Dense(neurons, activation='elu', kernel_initializer='he_normal'))
                flag = 1
    model.add(keras.layers.Dense(3))
    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=0.001),metrics=[coeff_determination])
    return model

In [165]:
#callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint("IKmodel.h5", save_best_only=True)
early_stopping = keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
lr_schedule = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)

In [166]:
#grid used in RandomSearchCV
grid = {
    'neurons' : [10, 30, 50],
    'layrs' : [5, 10, 20, 30],
    'gap' : [1, 2, 3]
}

In [167]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(model_development)
rnd_search_cv = RandomizedSearchCV(keras_reg, grid, n_iter=1, cv=3)

In [168]:
rnd_search_cv.fit(Xtrain, Ytrain, epochs=100, validation_data=(Xval, Yval),
callbacks=[checkpoint_cb, early_stopping, lr_schedule])

C:\Anaconda\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
2513/2513 [==============================] - 6s 2ms/step - loss: 339.6882 - coeff_determination: 0.9355 - val_loss: 122.1558 - val_coeff_determination: 0.9770
Epoch 2/100
2492/2513 [============================>.] - ETA: 0s - loss: 115.8780 - coeff_determination: 0.9781

KeyboardInterrupt: 